# Create Airbnb Athena Schemas

In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [3]:
ingest_create_athena_table_tsv_passed = False

In [4]:
%store -r ingest_create_athena_db_passed

In [5]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
print(ingest_create_athena_db_passed)

True


In [7]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [8]:
# Set paths to S3 Buckets
s3_listings_csv = "s3://aurelia-resort-data/airbnb/listings"
s3_neighbourhoods_csv = "s3://aurelia-resort-data/airbnb/neighbourhoods"
s3_reviews_csv = "s3://aurelia-resort-data/airbnb/reviews"

In [9]:
from pyathena import connect

In [10]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

## Create Listings Schema

In [11]:
# Set Athena parameters
database_name = "travel_airbnb"
table_listings = "listings_csv"

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [41]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
        id string,
        listing_url string,
        scrape_id string,
        last_scraped string,
        source string,
        name string,
        description string,
        neighborhood_overview string,
        picture_url string,
        host_id string,
        host_url string,
        host_name string,
        host_since string,
        host_location string,
        host_about string,
        host_response_time string,
        host_response_rate string,
        host_acceptance_rate string,
        host_is_superhost string,
        host_thumbnail_url string,
        host_picture_url string,
        host_neighbourhood string,
        host_listings_count string,
        host_total_listings_count string,
        host_verifications string,
        host_has_profile_pic string,
        host_identity_verified string,
        neighbourhood string,
        neighbourhood_cleansed string,
        neighbourhood_group_cleansed string,
        latitude string,
        longitude string,
        property_type string,
        room_type string,
        accommodates string,
        bathrooms string,
        bathrooms_text string,
        bedrooms string,
        beds string,
        amenities string,	
        price string,
        minimum_nights string,	
        maximum_nights string,
        minimum_minimum_nights string,
        maximum_minimum_nights string,
        minimum_maximum_nights string,
        maximum_maximum_nights string,
        minimum_nights_avg_ntm string,
        maximum_nights_avg_ntm string,
        calendar_updated string,
        has_availability string,
        availability_30 string,
        availability_60 string,
        availability_90 string,
        availability_365 string,
        calendar_last_scraped string,	
        number_of_reviews string,
        number_of_reviews_ltm string,	
        number_of_reviews_l30d string,
        first_review string,
        last_review string,
        review_scores_rating string,	
        review_scores_accuracy string,
        review_scores_cleanliness string,
        review_scores_checkin string,
        review_scores_communication string,
        review_scores_location string,
        review_scores_value string,
        license string,
        instant_bookable string,
        calculated_host_listings_count string,	
        calculated_host_listings_count_entire_homes string,	
        calculated_host_listings_count_private_rooms string,
        calculated_host_listings_count_shared_rooms string,
        reviews_per_month string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '\\"',
   'escapeChar' = '\\\\'
   )
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1');""".format(
    database_name, table_listings, s3_listings_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS travel_airbnb.listings_csv (
        id string,
        listing_url string,
        scrape_id string,
        last_scraped string,
        source string,
        name string,
        description string,
        neighborhood_overview string,
        picture_url string,
        host_id string,
        host_url string,
        host_name string,
        host_since string,
        host_location string,
        host_about string,
        host_response_time string,
        host_response_rate string,
        host_acceptance_rate string,
        host_is_superhost string,
        host_thumbnail_url string,
        host_picture_url string,
        host_neighbourhood string,
        host_listings_count string,
        host_total_listings_count string,
        host_verifications string,
        host_has_profile_pic string,
        host_identity_verified string,
        neighbourhood string,
        neighbourhood_cleansed string,
        neighbourhood_group_cleans

In [42]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [43]:
statement = """SELECT * FROM {}.{}
            LIMIT 5;""".format(
    database_name, table_listings
)

print(statement)

SELECT * FROM travel_airbnb.listings_csv
            LIMIT 5;


In [44]:
df = pd.read_sql(statement, conn)
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3686,https://www.airbnb.com/rooms/3686,20221220053924,2022-12-20,city scrape,Vita's Hideaway,IMPORTANT NOTES<br />* Carefully read and be s...,We love that our neighborhood is up and coming...,https://a0.muscache.com/pictures/61e02c7e-3d66...,4645,...,4.83,3.86,4.62,,f,1,0,1,0,0.55
1,196429,https://www.airbnb.com/rooms/196429,20221220053924,2022-12-20,city scrape,"Takoma Garden apt-Lush yard,pets ok",Private Garden apt in SF home with Full size b...,Small town feel in big city. Takoma Park is a...,https://a0.muscache.com/pictures/1768756/9d738...,956417,...,5.0,4.6,4.2,,f,1,1,0,0,0.06
2,198685,https://www.airbnb.com/rooms/198685,20221220053924,2022-12-20,city scrape,Private room in central location.,"Welcome, I'm happy to share my home with you. ...",The neighborhood is quiet for the most part bu...,https://a0.muscache.com/pictures/06e92870-bb6e...,966613,...,4.82,4.45,4.68,,f,1,0,1,0,1.12
3,201116,https://www.airbnb.com/rooms/201116,20221220053924,2022-12-20,city scrape,Victorian Haven - Private Basement!,<b>The space</b><br />Come stay with us in my ...,,https://a0.muscache.com/pictures/1841088/7148a...,983611,...,None,None,None,None,None,None,None,None,None,None
4,"Favorite movies for now are""The Women",""" ""Up",""" ""Blueberry Nights""; Art by Jacob Lawrence",Salvador Dali,Frida Kahlo; Music by Prince. Stevie,Ravel; I listen to Pop,Rock and Gospel and more. History art and sp...,"'phone'],t,t,,Takoma",Brightwood,"Manor Park,,38.97291,-77.02063,Entire rental ...",...,None,None,None,None,None,None,None,None,None,None


## Create Neighbourhoods Schema

In [17]:
# Set Athena parameters
table_neighbourhoods = "neighbourhoods_csv"

In [18]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
        neighbourhood_group string,
        neighbourhood string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '\"',
   'escapeChar' = '\\\\'
   )
STORED AS TEXTFILE
LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1');""".format(
    database_name, table_neighbourhoods, s3_neighbourhoods_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS travel_airbnb.neighbourhoods_csv (
        neighbourhood_group string,
        neighbourhood string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"',
   'escapeChar' = '\\'
   )
STORED AS TEXTFILE
LOCATION 's3://aurelia-resort-data/airbnb/neighbourhoods'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1');


In [19]:
pd.read_sql(statement, conn)

""


In [20]:
statement = """SELECT * FROM {}.{}
            LIMIT 5;""".format(
    database_name, table_neighbourhoods
)

print(statement)

SELECT * FROM travel_airbnb.neighbourhoods_csv
            LIMIT 5;


In [21]:
df = pd.read_sql(statement, conn)
df.head()

,neighbourhood_group,neighbourhood
0,,"Brightwood Park, Crestwood, Petworth"
1,,"Brookland, Brentwood, Langdon"
2,,"Capitol Hill, Lincoln Park"
3,,"Capitol View, Marshall Heights, Benning Heights"
4,,"Cathedral Heights, McLean Gardens, Glover Park"


## Create Reviews Schema

In [22]:
# Set Athena parameters
table_reviews = "reviews_csv"

In [23]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [24]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
        listing_id string,
        id string,
        date string,
        reviewer_id string,
        reviewer_name string,
        comments string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '\"',
   'escapeChar' = '\\\\'
   )
STORED AS TEXTFILE
LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1');""".format(
    database_name, table_reviews, s3_reviews_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS travel_airbnb.reviews_csv (
        listing_id string,
        id string,
        date string,
        reviewer_id string,
        reviewer_name string,
        comments string
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"',
   'escapeChar' = '\\'
   )
STORED AS TEXTFILE
LOCATION 's3://aurelia-resort-data/airbnb/reviews'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1');


In [25]:
pd.read_sql(statement, conn)

""


In [26]:
statement = """SELECT * FROM {}.{}
            LIMIT 5;""".format(
    database_name, table_reviews
)

print(statement)

SELECT * FROM travel_airbnb.reviews_csv
            LIMIT 5;


In [27]:
df = pd.read_sql(statement, conn)
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,28717282,611311066,2020-02-26,144251887,Danny,"This was our last stop in our US road trip, ha..."
1,28717282,612497481,2020-02-29,330825082,William,10/10 would stay here again!
2,28717282,613713908,2020-03-02,148439563,Loy,Very nice
3,28717282,615207226,2020-03-07,300366116,Sarah,Apartment is easily accessed from many histori...
4,28717282,616756958,2020-03-10,66650026,Nadia,Jamie's place was excellent! It was incredibly...


In [28]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head()

,tab_name
0,listings_csv
1,neighbourhoods_csv
2,reviews_csv


In [29]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


In [30]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_tsv_passed             -> False


## Release Resources

In [31]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [32]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>